<a href="https://colab.research.google.com/github/Yunpei24/BigDataBase/blob/main/Activite2_Question2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ACTIVITE2

Installation du JDK

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Création de la variable d'environnement <JAVA_HOME> pour situer l'emplacement d'installationde Java 

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Installation du framework Hadoop

Téléchargement depuis les archives de la fondation Apache

In [3]:
!wget https://archive.apache.org/dist/hadoop/common/hadoop-3.3.0/hadoop-3.3.0.tar.gz

--2023-01-16 14:56:08--  https://archive.apache.org/dist/hadoop/common/hadoop-3.3.0/hadoop-3.3.0.tar.gz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 500749234 (478M) [application/x-gzip]
Saving to: ‘hadoop-3.3.0.tar.gz’

hadoop-3.3.0.tar.gz 100%[===================>] 477.55M  1.20MB/s    in 10m 52s 

2023-01-16 15:07:00 (750 KB/s) - ‘hadoop-3.3.0.tar.gz’ saved [500749234/500749234]



Extraction de l'archive

In [ ]:
!tar -xzvf hadoop-3.3.0.tar.gz

Copie du dossier extrait dans l'emplacement <user/local>

In [5]:
!cp -r hadoop-3.3.0/ /usr/local/

## Programmation de tâches distribuées avec <i>MapReduce</i>

Création d'un repertoire <myinput> pour contenir le jeu de données à tester durant cet exercice e d'un second pour les résultats du traitement distribué

In [6]:
!mkdir -p ~/myinput
!mkdir -p ~/myoutput

Télachargement du jeu de données dans le fichier <u>purchases.txt</u>

In [7]:
!curl -L -o 'purchases.txt' 'https://drive.google.com/u/0/uc?id=1NS-PSXW8bSNpzFH4XRbtmMnMGhXBdYy6&export=download&confirm=t'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  201M  100  201M    0     0  60.3M      0  0:00:03  0:00:03 --:--:-- 69.7M


Déplacement du fichier <u>purchases.txt</u> dans le répertoire <myinput>

In [8]:
!mv purchases.txt ~/myinput/purchases.txt

Vérification que les fichiers ont été bien copiés

In [9]:
!ls /usr/myinput

ls: cannot access '/usr/myinput': No such file or directory


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Affichage des premiers lignes du fichier. Le format des enregistrement est le suivant:
<table border='1'><tr>
<td>Date</td><td>Heure</td><td>Magasin</td><td>Produit</td><td>Montant</td><td>Moyen_de_paiement</td>
</tr></table>
La tabulation <b>\t</b> est utilisée comme séparateur de colonne ✅

In [11]:
!head -5  ~/myinput/purchases.txt

2012-01-01	09:00	San Jose	Men's Clothing	214.05	Amex
2012-01-01	09:00	Fort Worth	Women's Clothing	153.57	Visa
2012-01-01	09:00	San Diego	Music	66.08	Cash
2012-01-01	09:00	Pittsburgh	Pet Supplies	493.51	Discover
2012-01-01	09:00	Omaha	Children's Clothing	235.63	MasterCard


#QUESTION2 : Le chiffre d'affaire réalisé selon les jours de la semaine

In [30]:
#!/usr/bin/python
import sys
from datetime import date
# Mettez vos instructions ...
weekdayDic = {
    0 : "Monday",
    1 : "Tuesday",
    2 : "Wednesday",
    3 : "Thursday",
    4 : "Friday",
    5 : "Saturday",
    6 : "Sanday"
}
for line in sys.stdin:
  data = line.strip().split("\t")
  if len(data) == 6:
    dates, time, store, item, amount, payment = data
    Day = dates.strip().split("-")
    if len(Day) == 3:
      year, month, day = Day
      weeknum = date(int(year), int(month), int(day)).weekday()

    print(weekdayDic[weeknum], "\t", amount)

In [31]:
# All cell codes are stored in a List variable "In"
with open('/content/mapper.py', 'w') as f:
  f.write(In[30]) 
f.close()

In [32]:
!chmod u+rwx /content/mapper.py

Test du traitement de la phase <map> sur quelques enregistrements

In [33]:
!head -10 ~/myinput/purchases.txt | python3 /content/mapper.py

Sanday 	 214.05
Sanday 	 153.57
Sanday 	 66.08
Sanday 	 493.51
Sanday 	 235.63
Sanday 	 247.18
Sanday 	 379.6
Sanday 	 296.8
Sanday 	 25.38
Sanday 	 213.88


In [34]:
#!/usr/bin/python
from operator import itemgetter
import sys
# Mettez vos instructions ...
salesTotal = 0
oldKey = None

for line in sys.stdin:
  data = line.strip().split("\t")
  if len(data) != 2:
    continue
  
  thisKey, thisSale = data
  if oldKey and oldKey != thisKey:
    print(oldKey, "\t", str(salesTotal))
    salesTotal = 0
    
  oldKey = thisKey
  salesTotal += float (thisSale)

if oldKey != None:
  print(oldKey, "\t", str(salesTotal))

In [35]:
# All cell codes are stored in a List variable "In"
with open('/content/reducer.py', 'w') as f:
  f.write(In[34]) 
f.close()

In [36]:
!chmod u+rwx /content/reducer.py

In [37]:
!head -20 ~/myinput/purchases.txt | python3 /content/mapper.py | sort | python3 /content/reducer.py

Sanday  	 5004.43


In [ ]:
!rm -r ~/myoutput
!/usr/local/hadoop-3.3.0/bin/hadoop jar /usr/local/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar -input ~/myinput -output ~/myoutput -file /content/mapper.py  -file /content/reducer.py  -mapper 'python mapper.py'  -reducer 'python reducer.py'

In [39]:
!tail -n 30 ~/myoutput/part-00000

Friday  	 147414929.5000001
Monday  	 150364112.0699944
Sanday  	 150296795.4699973
Saturday  	 147410177.57000166
Thursday  	 147353780.5700033
Tuesday  	 147246658.13999802
Wednesday  	 144371499.94000185
